# Call Center

### Introduction 

The date is 27 April, 2015. A call center which specializes in the sale of glittery merchandise advertised in late-night infomercials is having some doubts about an important business decision. Lacking the on-site expertise required to make sense of their data, they have hired you to examine the problem and lend your expert advise. You'll find that data in the file `CallCenter.csv`.

In the previous year, the company which has hired you (let's call it Company A) acquired another company (Company B). Since the acquisition, calls for Company B have begun migrating into Company A's call center. Today, this migration is 50% complete; half of the calls for Company B appear in the dataset. The other half will be migrated in the coming months.

Your task is to use this information to predict the optimal number of employees for the call center to employ. To do this, you will program a simulator of the call center. Once the simulation is written, a simple modification will allow us to predict what the sales data will look like upon the completion of the migration of Company B. This will inform the company's decisions about how many employees to hire.

## Part I: Exploring the data

### Load the data and get familiar with it
Let's begin by examining the data to see what we can find. Load the data into a pandas dataframe using the `read_csv()` function. Print out a list of the column headings as well as the first 10 rows of data. How many rows of data are in this dataset?

In [ ]:
import pandas as pd

In [ ]:
# Read the data 
df = pd.read_csv("CallCenter.csv.zip", compression='zip', header=0, sep=',', quotechar='"')

In [ ]:
# Print number of columns and first 10 rows
df.head(10)

### Examine the numeric columns

There are two columns we are particularly interested in: Credit Transfer and Total Time with Agent. Draw a histogram of the data in each of these columns. In your plots, can you identify the fingerprint of Company A and Company B?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df["Total Time with Agent"] = df["Total Time with Agent"] * 60

In [ ]:
import numpy as np


ct_np = np.array(df["Credit Transfer"].tolist())
tta_np = np.array(df["Total Time with Agent"].tolist())

print("Credit Transfer {} {} {} {} {}".format(ct_np.mean(),ct_np.var(),ct_np.std(),ct_np.min(),ct_np.max()))
print("Total Time with Agent {} {} {} {} {}".format(tta_np.mean(),tta_np.var(),tta_np.std(),tta_np.min(),tta_np.max()))

In [ ]:
# Plot histograms
#Complete Data
ct = df["Credit Transfer"].tolist()
tta = df["Total Time with Agent"].tolist()

plt.subplot(2,1,1)
plt.hist(ct,normed=True)
plt.title("Distribution of Credit Transfer")

plt.subplot(2,1,2)
plt.title("Distribution of Total Time with Agent")
plt.hist(tta,normed=True)

plt.tight_layout()

### Parse dates as datetime object

We know that Company A acquired Company B sometime towards the end of 2014. Since then, calls for Company B have been migrated in batches into the call center. Can we find out when this began? Let's begin by parsing the 'Call Date' column in order to create a new column in our dataframe which holds Python `datetime` objects. Check out the [`to_datetime`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) function.

In [ ]:
import datetime


In [ ]:
# Create datetime column
df["Call Date Converted"] = pd.to_datetime(df['Call Date'], format="%d %B %Y")

In [ ]:
df.head(20)

### Number of calls per day

Let's examine the number of calls per day. Use the pandas `groupby()` function together with `size()` to count how many calls were answered each day. We know that the calls from Company B started coming in sometime in the latter half of 2014. Draw a plot of number of calls per day vs. date ranging from 1 June 2014 to 1 January 2015. You may want to use a "boolean array" for filtering.

In [ ]:
# Plot calls per day
data=df.groupby(['Call Date Converted']).size().to_frame('Calls').reset_index()
data.columns =  ["Call Date","Total Calls"]
data.plot()

In [ ]:
data[325:330]

Adjust the date window to identify two key dates: 
1. When did the migration begin? 
2. When did the most recent batch of calls from Company B come to our call center? It is on this date that the migration is 50% complete.

In [ ]:
# Find key dates
date = data[115:116].iloc[0]["Call Date"]
date = datetime.datetime(2014,9,6)
print("the migration began at {}".format(date))

fifty_percent_date = data[327:328].iloc[0]["Call Date"]
print("the 50% migration completed on {}".format(fifty_percent_date))

## Part II: Modeling the data

Use the information from the above plot to draw histograms for 'Credit Transfer' and 'Total Time with Agent' pertaining strictly to Company A.

In [ ]:
from scipy import stats
# Draw histograms
before_mig_df = df[df['Call Date Converted'] < date]

ct = before_mig_df["Credit Transfer"].tolist()
tta = before_mig_df["Total Time with Agent"].tolist()

plt.subplot(1,2,1)
plt.hist(ct,normed=True , bins = 100)
mu = before_mig_df["Credit Transfer"].mean()
std = before_mig_df["Credit Transfer"].std()
plt.hist(np.random.normal(mu,std, size = 1000000), bins = 1000, normed = True,alpha = 0.5)

plt.subplot(1,2,2)
plt.hist(tta,normed=True , bins = 100)
mu = before_mig_df["Total Time with Agent"].mean()
std = before_mig_df["Total Time with Agent"].std()
plt.hist(np.random.normal(mu,std, size = 1000000), bins = 1000, normed = True,alpha=0.5)

plt.show()

### Total Time with Agent, prior to acquisition

From the preceding section, we can see that the Total Time with Agent distribution prior to the acquisition appears to be normal. Compute the mean and standard deviation for this distribution. Write a simulator which produces data with the same distribution. Plot a histogram of the original data and simulated data on the same plot. Use the `alpha` parameter of the `hist()` function to adjust transparancy so that both distributions can be seen.

In [ ]:
before_mig_df.hist(figsize=(16,20),bins=100)

In [ ]:
import numpy as np

### Total Time with Agent - following 50% migration

Let's consider the distribution of Total Time with Agent following the most recent batch of migrated calls. 

When a new call comes in, with a certain probability $p$, the call will be for Company A. Otherwise, it is for Company B. To begin with, let's guess that $p=0.5$.

If the call is for Company A, we can assume that the distribution of Total Time with Agent will be the same as it was prior to the migration.

If the call is for Company B, it will follow a different distribution with its own (so far unknown) mean and standard deviation. Let's start with a guess. Say $\bar{x}_B=30$ and $\sigma_B=2$.

Write a function which simulates a call time based on the above criteria including the initial guesses. Draw a histogram of the results of your simulation on top of the histogram of the actual data. You will find that they do not match too well. Tweak the variables we have guessed ($p$, $\bar{x}_B$, and $\sigma_B$) until you have identified values which better match the observed distribution.

In [ ]:
print(mu)
print(std)

In [ ]:
plt.hist(np.concatenate((np.random.normal(loc=mu,scale=std, size = 1000000),np.random.normal(loc=30*60,scale=2*60, size = 1000000))))

In [ ]:
# Write simulation function here
post_50_df = df[df['Call Date Converted'] >= fifty_percent_date]
post_50_tta = post_50_df["Total Time with Agent"].tolist()
post_50_tta

In [ ]:
#probability of call coming to A or B = 0.5 - Binom

plt.hist(tta,normed=True , bins = 100)
plt.hist(post_50_tta,normed=True , bins = 100)
#Company B - normal distribution mean and std deviation , 30 and 2
plt.hist(np.random.normal(loc=30*60,scale=2*60, size = 1000000), bins = 1000, normed = True,alpha=0.5)
#plt.hist(np.random.normal(loc=28,scale=6, size = 1000000), bins = 1000, normed = True,alpha=0.5)

my_list = np.concatenate((np.random.normal(loc=mu,scale=std, size = 1000000),np.random.normal(loc=28.4*60,scale=3.8*60, size = 1000000)))

plt.hist(my_list,bins = 1000, normed = True,alpha=0.5)

plt.show()

In [ ]:
random_nrm_data_tta_compA = np.random.normal(loc=mu,scale=std, size = 1000000)
random_nrm_data_tta_compB = np.random.normal(loc=28.4*60,scale=3.8*60, size = 1000000)

randomsamplesA = []
randomsamplesB = []
for i in range(1000001):
    outcome = np.random.binomial(1, .57)
    #1 assumed to be company A
    if (outcome == 1):
        randomsampleA = np.random.choice(random_nrm_data_tta_compA)
        randomsamplesA.append(randomsampleA)
    else:
        randomsampleB = np.random.choice(random_nrm_data_tta_compB)
        randomsamplesB.append(randomsampleB)

plt.hist(randomsamplesA+randomsamplesB,normed=True , bins = 100)
#plt.hist(randomsamplesB,normed=True , bins = 100)
plt.show()

In [ ]:
print(np.asarray(randomsamplesB).mean())
print(np.asarray(randomsamplesB).std())

In [ ]:
# Plot histograms of original and simulated data

### Credit Transfer

Let's check if the Total Time with Agent is correlated to the Credit Transfer. First, use the `plot()` function to draw a scatterplot of these two variables. Put Total Time with Agent on the x axis and Credit Transfer on the y axis.

In [ ]:
# Scatterplot
df.plot.scatter(x="Total Time with Agent", y="Credit Transfer")

Next, let's use the `np.polyfit()` function to fit a best fit line through this data. To get a best fit line, set the `deg` parameter of `polyfit` to 1. Its outputs will be the slope and intercept of a line. Plot the best fit line on top of the scatterplot.

In [ ]:
# Compute slope and intercept + # Scatterplot with best fit line
fig, ax = plt.subplots()
fit = np.polyfit(x=df.loc[:,"Total Time with Agent"], y=df.loc[:,"Credit Transfer"], deg=1)
ax.plot(df.loc[:,"Total Time with Agent"], fit[0] * df.loc[:,"Total Time with Agent"] + fit[1], color='red')
ax.scatter(x=df.loc[:,"Total Time with Agent"], y=df.loc[:,"Credit Transfer"])

fig.show()

Notice that the datapoints are still scattered above and below the best fit line. Let's examine the spread of these datapoints. 

For each point, compute the error of the fit. The error is the true value of Credit Transfer subtract the fitted value from the model, or:

```
error = (True Credit Transfer) - (slope*Total Time with Agent + intercept)
```

Plot a histogram of this error and compute its mean and standard deviation. 

In [ ]:
print(fit[0])
print(fit[1])

In [ ]:
# Plot histogram of error
error = df.loc[:,"Credit Transfer"] - (fit[0]*df.loc[:,"Total Time with Agent"]  + fit[1])
error_df = error.to_frame(name="Error")
error_df.hist(normed=True,bins=100)

We now will return to the simulation we wrote for Total Time with Agent. 

Using the mean and standard deviation of the error as well as the slope and intercept of the best fit line, improve your simulation to provide not only Total Time with Agent but also Credit Transfer following the 50% migration date. Once again, plot the histograms on the same plot with two colors to see that they line up.

In [ ]:
# Simulate both credit transfer and total time with agent
random_nrm_data_tta_compA = np.random.normal(loc=mu,scale=std, size = 1000000)
random_nrm_data_tta_compB = np.random.normal(loc=28.4*60,scale=3.8*60, size = 1000000)
random_nrm_data_ct_error = np.random.normal(loc=error_df.mean(),scale=error_df.std(), size = 1000000)

randomsamplesCT = []
for i in range(100000):
    outcome = np.random.binomial(1, .57)
    #1 assumed to be company A
    if (outcome == 1):
        randomsampleA = np.random.choice(random_nrm_data_tta_compA)
        ct_error = np.random.choice(random_nrm_data_ct_error)
        ctwitherror = (fit[0]*randomsampleA+fit[1])+ct_error
        randomsamplesCT.append( ctwitherror)
    else:
        randomsampleB = np.random.choice(random_nrm_data_tta_compB)
        ct_error = np.random.choice(random_nrm_data_ct_error)
        ctwitherror = (fit[0]*randomsampleB+fit[1])+ct_error
        randomsamplesCT.append(ctwitherror)

        
randomsamplesA = []
randomsamplesB = []
for i in range(1000001):
    outcome = np.random.binomial(1, .57)
    #1 assumed to be company A
    if (outcome == 1):
        randomsampleA = np.random.choice(random_nrm_data_tta_compA)
        randomsamplesA.append(randomsampleA)
    else:
        randomsampleB = np.random.choice(random_nrm_data_tta_compB)
        randomsamplesB.append(randomsampleB)

#plt.hist(randomsamplesA+randomsamplesB,normed=True , bins = 100)
plt.hist(randomsamplesCT,normed=True , bins = 100)
plt.show()

In [ ]:
# Plot histogram of credit transfer

post_50_ct = post_50_df["Credit Transfer"].tolist()
plt.hist(post_50_ct,normed=True , bins = 100)
plt.hist(randomsamplesCT,normed=True , bins = 100,alpha=0.5)
plt.show()


### Call times

Let's find out the average amount of time between calls. To begin, modify the `datetime` column to include the call time. This will allow us to subtract two `datetime` objects to get a time difference.

*Hint:* You may find [this question](http://stackoverflow.com/questions/17978092/combine-date-and-time-columns-using-python-pandas) on stackoverflow useful.

In [ ]:
# Convert call time to datetime
df["Call Date Time"] = pd.to_datetime(df['Call Date'] + ' ' + df['Call Time'],format="%d %B %Y %H:%M:%S")
df

Now that we have the call time included in our `datetime`, let's compute the difference between successive columns. For this we'll use the pandas `diff()` function. Create a new column which stores the number of seconds that elapsed since the last call. 

*Hint:* What is the type of the object created when you subtract one `datetime` object from another? Look for a method to convert this object into a number of seconds.

In [ ]:
# Compute difference between call times
df["Call Date Diff"] = pd.DataFrame(df["Call Date Time"]).diff(periods=1, axis=0)
df["Call Date Total Seconds"] = df["Call Date Diff"].apply(lambda x: x.total_seconds())

What is the average time between calls? Compute the mean. Draw a histogram of the time between calls in seconds. On the same plot, draw the PDF of a corresponding exponential distribution.

In [ ]:
from scipy.stats import expon

df["Call Date Total Seconds"].hist(bins=100,normed=True)

exps = []
for i in range(1000):
    exps.append(np.random.exponential(df["Call Date Total Seconds"].mean()))

plt.hist(exps,bins=100,normed=True)

mu = df["Call Date Total Seconds"].mean()
t = np.linspace(1,df["Call Date Total Seconds"].max())

def f(t,mu):
    return 1/mu* np.exp(-t/mu)

plt.plot(t, f(t,mu))

plt.show()



In [ ]:
post_50_df["Call Date Time"] = pd.to_datetime(post_50_df['Call Date'] + ' ' + post_50_df['Call Time'],format="%d %B %Y %H:%M:%S")
post_50_df["Call Date Diff"] = pd.DataFrame(post_50_df["Call Date Time"]).diff(periods=1, axis=0)
post_50_df["Call Date Total Seconds"] = post_50_df["Call Date Diff"].apply(lambda x: x.total_seconds())

In [ ]:
# Draw data histogram and exponential pdf
post_50_df["Call Date Total Seconds"].hist(bins=100,normed=True)

exps = []
for i in range(1000):
    exps.append(np.random.exponential(post_50_df["Call Date Total Seconds"].mean()))

plt.hist(exps,bins=100,normed=True)

mu = post_50_df["Call Date Total Seconds"].mean()
t = np.linspace(1,post_50_df["Call Date Total Seconds"].max())


plt.plot(t, f(t,mu))

plt.show()



What are the average number of calls per day following the latest migration?

In [ ]:
# Average number of calls per day
grouped_data = post_50_df.groupby("Call Date Converted").size().to_frame('Total Calls').reset_index()
sum(grouped_data["Total Calls"])/len(grouped_data)

Of these, how many are due to Company A and how many are due to Company B? *Hint:* Use the value for $p$ you discovered above.

In [ ]:
# Average calls for Company A and for Company B
totalcallsA = []
totalcallsB = []
for j in range(0,10000):
    calls = []
    for i in range(0,2405):
        calls.append(np.random.binomial(1, .57))
    totalcallsA.append(sum(calls))
    totalcallsB.append(2405-sum(calls))
print(sum(totalcallsA)/10000)
print(sum(totalcallsB)/10000)



In [ ]:
np.random.binomial(2405,.57)

How many calls do you expect per day after the migration of Company B is complete?

In [ ]:
#  Total number of expected calls after migration
#Average calls to company B are 1034 at 50%, approx total number of calls after migration 2000

What would be the average time between calls (in seconds) for this many customers?

In [ ]:
# Average time between calls 
post_50_df["Call Date Total Seconds"].fillna(value=0,inplace=True)
average_time_between_calls_50 = post_50_df["Call Date Total Seconds"].sum()/post_50_df.shape[0]

In [ ]:
86400/3430

In [ ]:
#Probability of calls to A 
1367/3430

## Combined simulation

Create a function that simulates a day's worth of calls. Take `number_of_employees` as an input. Use a list with length equal to number of employees to track whether a given employee is busy on a call. If all employees are busy, assume the call is lost. Compute the total credit transfer for a day with 10 employees. Also compute the number of calls dropped.

In [ ]:
# Write a function to simulate a days worth of calls
def total_credit_transfer_per_day(number_of_employees,
                                  time_between_calls,
                                  mean_of_ttwa_companyA,
                                  std_dev_of_ttwa_companyA,
                                  mean_of_ttwa_companyB,
                                  std_dev_of_ttwa_companyB,
                                  mean_of_error,
                                  std_dev_of_error,
                                  slope,
                                  intercept,
                                  probability_of_assignment
                                 ):
    number_of_employees_available = number_of_employees
    
    #On call list
    ttwa_of_on_call_employee = []
    
    time = 0
    call_drop = 0
    credit_transfer = 0
    #f= open("debug.txt","w+")
    while(time < 86400):
        #Time of next call
        time +=np.random.exponential(time_between_calls)
        
        #Filter on call list to store number of employees post completed calls
        filterlist = [elem for elem in ttwa_of_on_call_employee if elem <= time]
        employees_available_post_call = len(filterlist)
        number_of_employees_available += employees_available_post_call
        #Filter on call list to remove completed calls
        ttwa_of_on_call_employee = [elem for elem in ttwa_of_on_call_employee if elem > time]
        
        if (number_of_employees_available > 0):
            companyA = np.random.binomial(1,probability_of_assignment)
            if (companyA):
                ttwa_sample = np.random.normal(loc=mean_of_ttwa_companyA,scale=std_dev_of_ttwa_companyA)        
            else:
                ttwa_sample = np.random.normal(loc=mean_of_ttwa_companyB,scale=std_dev_of_ttwa_companyB)
            #Time of call + norm random of ttwa
            ttwa_of_on_call_employee.append((ttwa_sample) + time)
            ct = (slope*ttwa_sample+intercept)+np.random.normal(loc=mean_of_error,scale=std_dev_of_error)
            credit_transfer += ct
            number_of_employees_available = number_of_employees_available - 1
        else:
            call_drop += 1
        #f.write("No of employees:{},Time:{},TTWA List:{} \r\n".format(number_of_employees_available,time,ttwa_of_on_call_employee))
    #f.close()
    return (call_drop,credit_transfer)

In [ ]:
np.random.normal(loc=28.4*60,scale=3.8*60) + 1000

In [ ]:
# Simulate a day's worth of calls for 10 employees
total_credit_transfer_per_day(10,86400/3430,mu,std,28.4*60,3.8*60,error_df["Error"].mean(),error_df["Error"].std(),fit[0],fit[1],1367/3430)

In [ ]:
results

Use this simulator to make a plots of 1.) number of employees vs. Credit Transfer and 2.) number of employees vs. dropped calls. How many employees should the company hire to eliminate dropped calls?

In [ ]:
# Loop over number of employees and run simulator
results = []
for number_of_employees in range(0,100):
    call_drops = 0
    credit_transfer = 0
    item = total_credit_transfer_per_day(number_of_employees,86400/3430,mu,std,28.4*60,3.8*60,error_df["Error"].mean(),error_df["Error"].std(),fit[0],fit[1],1367/3430)
    results.append((number_of_employees ,item[0] , item[1] ))

In [ ]:
# Plot Credit Transfer and Dropped Calls vs. number of employees
new_df = pd.DataFrame(results,columns=["No of Employees","Call Drop","Credit Transfer"])
new_df

new_df.plot.scatter(x="No of Employees", y="Credit Transfer")
new_df.plot.scatter(x="No of Employees", y="Call Drop")

Use the simulator to generate profits at different numbers of employees.  Create a matrix of profits, with costs per employee increasing down the rows, and the number of employees increasing across the columns. 

Currently the call center optimizes NOT dropping calls, and they hire the minimum number of employees to ensure that no calls are dropped.  
If the company were to switch to profit maximization, at what cost per employee should the company start considering allow some dropped calls to save on costs?

In [ ]:
for i in range(8,36):
    new_df["Profit at {} USD per hour".format(i)] = new_df["Credit Transfer"] - (new_df["No of Employees"] * i*24)

In [ ]:
new_df

In [ ]:
new_df.plot()

In [ ]:
max_row = new_df["Profit at 8 USD per hour"].argmax()
new_df[max_row:max_row+1]